# Prey notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import neat
import matplotlib.pyplot as plt

import pickle
import multimodal_mazes
from tqdm import tqdm

## Ideas 

* Odour - constant but noisy. 
* Sound - reliable but infrequent.
* Rather than resetting the env to zero every step, you could decay it. E.g. env[:,:,:-1] *= 0.8 + blur. 
* Analysis: n_prey caught, speed, costs (e.g. movement vs food). 
* Evolve prey against different algorithms. Then, evolve predators against these prey.   

## Rule-based Agents

In [ ]:
# Hyperparameters 
size = 7
n_prey = 10
n_steps = 50
n_trials = 1000
pk = 5 # the width of the prey's Gaussian signal (in rc)

### Fitness vs noise

In [ ]:
# Fitness vs noise
from matplotlib import cm 
noises = np.linspace(start=0.0, stop=2.0, num=13)
policies = multimodal_mazes.AgentRuleBased.policies
results = np.zeros((len(noises), len(policies)))
cols = cm.get_cmap('plasma', len(policies))

# Test agents
for a, noise in enumerate(tqdm(noises)):

    for b, policy in enumerate(policies): 
        agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], policy=policy)
        fitness, _, _, _ = multimodal_mazes.eval_predator_fitness(n_trials=n_trials, size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk, n_steps=n_steps)

        results[a, b] = fitness

for b, policy in enumerate(policies): 
    plt.plot(noises, results[:,b], color=cols.colors[b], label=policy)

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Sensor Noise')
plt.legend()

In [ ]:
# Fitness vs noise AUC 
auc = np.trapz(y=results.T, x=noises, axis=1)
for b, _ in enumerate(policies): 
    ml, sl, _ = plt.stem(b, auc[b])
    ml.set_color(cols.colors[b])
    sl.set_color(cols.colors[b])
plt.xticks(range(len(policies)), policies, rotation='vertical')
plt.ylabel('AUC');

### Memory

In [ ]:
# Fitness vs noise
noises = np.linspace(start=0.0, stop=2.0, num=13)
policies = multimodal_mazes.AgentRuleBasedMemory.policies
alphas = np.linspace(start=0.0, stop=2.0, num=11)
results = np.zeros((len(noises), len(policies), len(alphas)))

# Test agents
for a, noise in enumerate(tqdm(noises)):

    for b, policy in enumerate(policies): 

        for c, alpha in enumerate(alphas):
            agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy=policy)
            agnt.alpha=alpha
            fitness, _, _, _ = multimodal_mazes.eval_predator_fitness(n_trials=n_trials, size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk, n_steps=n_steps)
            results[a, b, c] = fitness

In [ ]:
# Fitness vs noise
cols = ['#0ABAB5', 'xkcd:grey']
auc = np.trapz(y=results.T, x=noises, axis=2)
for b, policy in enumerate(policies): 
    plt.plot(alphas, auc[:,b], color=cols[b], label=policy)

plt.ylabel('AUC')
plt.xlabel(r"$\alpha$")
plt.legend()

## Plotting

In [ ]:
# agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy='Recurrent outputs')
agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy='Hidden skip')
agnt.alpha=2.0
time, path, prey_state, prey_locations = multimodal_mazes.predator_trial(size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk,n_steps=n_steps)
print(prey_state)

In [ ]:
# Plotting
from matplotlib import colors
prey_markers = ['P', 'X']

# Environment 
pk_hw = pk // 2  # half width of prey's Gaussian signal (in rc)
env = np.zeros((size, size, len(agnt.channels) + 1))
env[:, :, -1] = 1.0
env = np.pad(env, pad_width=((pk_hw, pk_hw), (pk_hw, pk_hw), (0, 0)))
plt.imshow(1 - env[:, :, -1], cmap="binary", alpha=0.25)

# Path
path = np.array(path)
cmap = colors.LinearSegmentedColormap.from_list(
    "", ["xkcd:teal blue", "xkcd:off white", "xkcd:coral"], N=n_steps
)
for t in range(len(path) - 1):
    plt.plot([path[t, 1], path[t + 1, 1]], [path[t, 0], path[t + 1, 0]], c=cmap(t), zorder=0)
    plt.scatter(path[t + 1, 1], path[t + 1, 0], s=30, color=cmap(t), zorder=1)

# Prey 
for n, prey_rc in enumerate(prey_locations):
    plt.scatter(prey_rc[1], prey_rc[0], color='k', alpha=0.5, marker=prey_markers[n % 2], zorder=2)

# Adjust axes 
plt.xlim([(pk//2) - 1, size + pk//2])
plt.ylim([size + pk//2, (pk//2) - 1]) 
plt.axis("off")


### WIP: Video

In [ ]:
from scipy import signal

pk_hw = pk // 2  # half width of prey's Gaussian signal (in rc)

# Create environment with track (1. and walls 0.)
env = np.zeros((size, size, len(agnt.channels) + 1))
env[:, :, -1] = 1.0
env = np.pad(env, pad_width=((pk_hw, pk_hw), (pk_hw, pk_hw), (0, 0)))

# Reset agent
agnt.location = np.array([pk_hw + (size // 2), pk_hw + (size // 2)])
agnt.sensor_noise_scale = noise
agnt.outputs *= 0.0
if agnt.type == "Hidden skip":
    agnt.memory = np.zeros_like(agnt.outputs)

# Define prey
k1d = signal.gaussian(pk, std=1)
k2d = np.outer(k1d, k1d)

rcs = np.stack(np.argwhere(env[:, :, -1]))
prey_rcs = np.random.choice(range(len(rcs)), size=n_prey, replace=False)
preys = []
for n in range(n_prey):
    preys.append(
        multimodal_mazes.AgentRandom(location=rcs[prey_rcs[n]], channels=[0, 0])
    )
    preys[n].state = 1  # free (1) or caught (0)
    preys[n].cues = n % 2  # channel for emitting cues

# Sensation-action loop
path = [list(agnt.location)]
prey_counter = np.copy(n_prey)
for time in range(n_steps):

    env[:, :, :-1] *= 0.0  # reset channels

    # Prey
    for prey in preys:
        if prey.state == 1:
            if (prey.location == agnt.location).all():  # caught
                prey.state = 0
                prey_counter -= 1

            else:  # free
                r, c = prey.location
                env[
                    r - pk_hw : r + pk_hw + 1, c - pk_hw : c + pk_hw + 1, prey.cues
                ] += np.copy(
                    k2d
                )  # emit cues

    # Apply edges
    for ch in range(len(agnt.channels)):
        env[:, :, ch] *= env[:, :, -1]

In [ ]:
cmap = colors.LinearSegmentedColormap.from_list(
    "", ["white", "xkcd:ultramarine"]
)

cmap1 = colors.LinearSegmentedColormap.from_list(
    "", ["white", "xkcd:magenta"]
)

In [ ]:
plt.imshow((cmap(env[:,:,0]) + cmap1(env[:,:,1]))/2)
plt.imshow(1 - env[:, :, -1], cmap="binary", alpha=0.25)
# Adjust axes 
plt.xlim([(pk//2) - 1, size + pk//2])
plt.ylim([size + pk//2, (pk//2) - 1]) 
plt.axis("off")
